In [5]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is the meaning of {word}")

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={
        "max_new_tokens": 250,
    }
)
llm.client.api_url = 'https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1'

chain = prompt | llm

chain.invoke({
    "word":"potato"
})

'?\nA: A starchy tuberous vegetable that is a staple food in many cultures, grown in rows in the ground and harvested when the plants die back. It is a popular food source and is used in a variety of dishes, including soups, salads, and fries.'